In [ ]:
import numpy as np
import os
import pandas as pd
import pathlib
import requests
import scanpy as sc
import seaborn as sns
import scvi

In [ ]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
SAMPLE_RAW_DATA_PATH = DATA_DIR / 'raw_data' / 'GSM5226574_C51ctr_raw_counts.csv.gz'

In [ ]:
adata = sc.read_csv(SAMPLE_RAW_DATA_PATH).T
adata

## Doublet removal

In [ ]:
# Filter genes expressed in less than 10 cells
sc.pp.filter_genes(adata, min_cells=10)

In [ ]:
# Keep only top 2000 highly variable genes
sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True, flavor='seurat_v3')

In [ ]:
scvi.model.SCVI.setup_anndata(adata)
model = scvi.model.SCVI(adata)
model.train()

In [ ]:
solo = scvi.external.SOLO.from_scvi_model(model, adata)
solo.train()

In [ ]:
df = solo.predict()
df['prediction'] = solo.predict(soft = False)

df

In [ ]:
df.groupby('prediction').count()

In [ ]:
df['dif'] = df.doublet - df.singlet
df

In [ ]:
sns.displot(df[df.prediction == 'doublet'], x = 'dif')

In [ ]:
doublets = df[(df.prediction == 'doublet') & (df.dif > 1)]
doublets

In [ ]:
adata = sc.read_csv(SAMPLE_RAW_DATA_PATH).T
adata

In [ ]:
adata.obs['doublet'] = adata.obs.index.isin(doublets.index)

In [ ]:
adata.obs

In [ ]:
adata = adata[~adata.obs.doublet]

In [ ]:
adata

## Preprocessing

In [ ]:
adata.var['mt'] = adata.var.index.str.startswith('MT-')

In [ ]:
response = requests.get('https://www.gsea-msigdb.org/gsea/msigdb/download_geneset.jsp?geneSetName=KEGG_RIBOSOME&fileType=json').json()
ribo_genes = response["KEGG_RIBOSOME"]['geneSymbols']

ribo_genes = np.array(ribo_genes)

In [ ]:
adata.var['ribo'] = adata.var.index.isin(ribo_genes)

In [ ]:
adata.var[adata.var.ribo == True]

In [ ]:
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribo'], percent_top=None, log1p=False, inplace=True)

In [ ]:
adata.obs

In [ ]:
adata.var.sort_values('n_cells_by_counts')

In [ ]:
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
adata.obs.sort_values('total_counts')

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo'], jitter=0.4, multi_panel=True)

In [ ]:
upper_lim = np.quantile(adata.obs.n_genes_by_counts, .98)
adata = adata[adata.obs.n_genes_by_counts < upper_lim]
adata = adata[adata.obs.pct_counts_mt < 20]
adata = adata[adata.obs.pct_counts_ribo < 2]
adata

## Normalization

In [ ]:
adata.X.sum(axis = 1)

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
adata.X.sum(axis = 1)

In [ ]:
sc.pp.log1p(adata) # change to log counts
adata.X.sum(axis = 1)

In [ ]:
adata.raw = adata

## Clustering

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000)
adata.var

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
adata = adata[:, adata.var.highly_variable]
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt', 'pct_counts_ribo'])
sc.pp.scale(adata, max_value=10)

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50)

In [ ]:
sc.pp.neighbors(adata, n_pcs=30)
adata.obsp['distances'].toarray()

In [ ]:
sc.tl.umap(adata)
sc.pl.umap(adata)

In [ ]:
sc.tl.leiden(adata, resolution=0.5)
adata.obs

In [ ]:
sc.pl.umap(adata, color=['leiden'])